In [26]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [30]:
rent_obj = []
all_property = []

for page in range(21,29):
    link = 'https://www.apartments.com/condos/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     page 1-5
#     results = soup.find_all('article', class_='placard placard-option has-header js-premiumplus')
#     page 5-28
#     print(soup)
    results = soup.find_all('article', class_='placard placard-option no-header js-premium')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
#         find_tag = result.find('a')['href']
        find_tag = result.find('a', class_="property-link")['href']
        print(find_tag)
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "property-address").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        try:
            lease_length = house_details.find('td', class_="leaseLength").text
        except:
            lease_length = 'null'
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Condos",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

https://www.apartments.com/200-pacific-coast-hwy-huntington-beach-ca-unit-328/xy1p1dn/
https://www.apartments.com/31423-coast-hwy-laguna-beach-ca-unit-66/6rmlt5s/
https://www.apartments.com/56250-las-rocas-mountain-center-ca/dxvq4ll/


KeyboardInterrupt: 

In [28]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,1 Bed,3 Beds,2 Beds,Studio,1 – 2 Beds,3 – 4 Beds,4 Beds,1 – 4 Beds
0,San Diego,Condos,CA,92108,"[{'bedroom_type': '1 Bed', 'price': '$1,75'}]","6780 Friars Rd, San Diego, CA 92108",https://www.apartments.com/6780-friars-rd-san-...,1 BA,640 Sq Ft,640,12 Month Lease,Available Now,"$1,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Los Angeles,Condos,CA,90291,"[{'bedroom_type': '3 Beds', 'price': '$4,85'}]","Los Angeles, CA 90291",https://www.apartments.com/27-s-venice-blvd-lo...,3 BAs,"1,500 Sq Ft","1,500",12 Month Lease,Available Now,NaN,"$4,85",NaN,NaN,NaN,NaN,NaN,NaN
2,San Jose,Condos,CA,95127,"[{'bedroom_type': '2 Beds', 'price': '$2,45'}]","247 N Capitol Ave, San Jose, CA 95127",https://www.apartments.com/247-n-capitol-ave-s...,1 BA,828 Sq Ft,828,12 Month Lease,Available Now,NaN,NaN,"$2,45",NaN,NaN,NaN,NaN,NaN
3,Los Angeles,Condos,CA,90068,"[{'bedroom_type': 'Studio', 'price': '$1,49'}]","Los Angeles, CA 90068",https://www.apartments.com/3767-fredonia-dr-lo...,1 BA,600 Sq Ft,600,12 Month Lease,Available Now,NaN,NaN,NaN,"$1,49",NaN,NaN,NaN,NaN
4,Huntington Beach,Condos,CA,92649,"[{'bedroom_type': 'Studio', 'price': '$1,70'}]","Huntington Beach, CA 92649",https://www.apartments.com/17088-st-andrews-ln...,1 BA,500 Sq Ft,500,12 Month Lease,Available Now,NaN,NaN,NaN,"$1,70",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Burbank,Condos,CA,91504,"[{'bedroom_type': '2 Beds', 'price': '$2,50'}]","Burbank, CA 91504",https://www.apartments.com/2318-n-keystone-st-...,1 BA,750 - 800 Sq Ft,750,12 Month Lease,Available Now,NaN,NaN,"$2,50",NaN,NaN,NaN,NaN,NaN
196,Irvine,Condos,CA,92620,"[{'bedroom_type': '3 Beds', 'price': '$3,00'}]","19 Christamon S, Irvine, CA 92620",https://www.apartments.com/19-christamon-s-irv...,2½ BAs,"1,416 Sq Ft","1,416",12 Month Lease,Available Now,NaN,"$3,00",NaN,NaN,NaN,NaN,NaN,NaN
197,Tustin,Condos,CA,92780,"[{'bedroom_type': '2 Beds', 'price': '$1,97'}]","Tustin, CA 92780",https://www.apartments.com/1716-nisson-rd-tust...,1 BA,"1,000 Sq Ft","1,000",12 Month Lease,Available Now,NaN,NaN,"$1,97",NaN,NaN,NaN,NaN,NaN
198,Whittier,Condos,CA,90601,"[{'bedroom_type': '3 Beds', 'price': '$2,40'}]","Whittier, CA 90601",https://www.apartments.com/5643-pickering-ave-...,1 BA,790 Sq Ft,790,,Available Now,NaN,"$2,40",NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q4-2020_Apartment.com_Condos_Data_page21-28.csv", index=False, header=True)